In [1]:
from train import train_encoder
import torch
from dataset import AudioSamplePairDataset
from train import transform_sample

/mnt/backup/ml/audio/venv/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
encoder = train_encoder()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 2, 64, 32]               8
       BatchNorm2d-2            [-1, 2, 64, 32]               4
               ELU-3            [-1, 2, 64, 32]               0
            Conv2d-4            [-1, 4, 32, 32]              28
       BatchNorm2d-5            [-1, 4, 32, 32]               8
               ELU-6            [-1, 4, 32, 32]               0
            Conv2d-7            [-1, 8, 32, 16]             104
       BatchNorm2d-8            [-1, 8, 32, 16]              16
               ELU-9            [-1, 8, 32, 16]               0
           Conv2d-10           [-1, 16, 16, 16]             400
      BatchNorm2d-11           [-1, 16, 16, 16]              32
              ELU-12           [-1, 16, 16, 16]               0
           Conv2d-13            [-1, 32, 16, 8]           1,568
      BatchNorm2d-14            [-1, 32

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/mnt/backup/ml/audio/venv/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/mnt/backup/ml/audio/venv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/mnt/backup/ml/audio/venv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/mnt/backup/ml/audio/dataset.py", line 29, in __getitem__
    return self.transform(val), self.transform(val), self.transform(contr_val)
  File "/mnt/backup/ml/audio/train.py", line 32, in transform_sample
    x = x.cuda()
  File "/mnt/backup/ml/audio/venv/lib/python3.8/site-packages/torch/cuda/__init__.py", line 163, in _lazy_init
    raise RuntimeError(
Run

In [ ]:
test_data = AudioSamplePairDataset(root_path='data/fma_small_samples', test=False, transform=transform_sample)

In [ ]:
t = [test_data[i][0] for i in range(1000)]
t_s = torch.stack(t)
t_e = encoder(t_s.cuda()).cpu()

In [ ]:
ok = 0
for tested_idx in range(1000):
    # tested_idx = 100
    x = test_data[tested_idx][1]
    x_s = x.unsqueeze(0)
    x_e = encoder(x_s.cuda()).cpu()
    dists = (x_e - t_e).abs().sum(dim=-1)
    # print((x_e - t_e[tested_idx]).abs().sum(dim=-1))
    top = torch.topk(dists, 20, largest=False)
    # print(top)
    if top.indices[0] == tested_idx:
        # print('OK')
        ok += 1
    # else:
    #     print('ERR')
print(ok / 1000)

In [ ]:
import IPython.display as ipd
ipd.Audio(test_data[0][0].squeeze(0), rate=22050)

In [ ]:
a = test_data[2][0]